In [1]:
import torch
from torch2trt import TRTModule

basic_name = "0422_Ep50_race_train"

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('{}_TRT.pth'.format(basic_name)))

<All keys matched successfully>

In [2]:
from utils.preprocess_utils import preprocess
import numpy as np

In [3]:
project = "0422school"
dataset = "A"
category = "race_train"

width = 224
height = 224

In [4]:
import ipywidgets
from IPython.display import display
import threading
import time
from utils.utils import channel12ToImgSize, creatProjectFolder, readInTagsFolderAndSort, getChannelValue, saveImage, bgr8_to_jpeg, imgXYToChannel12
from utils.ImageList import ImageList

In [5]:
global image_list
image_list = ImageList(project, category, dataset, random_hflip=False)

There is 5153 file in the project folder


In [6]:
train_view_widget = ipywidgets.Image(width=width, height=height)
count_widget = ipywidgets.IntText(description='count', disabled=True, value=len(image_list), style={'description_width': 'initial'})
trained_CH1_widget = ipywidgets.BoundedFloatText(description='CH1', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
trained_CH2_widget = ipywidgets.BoundedFloatText(description='CH2', value=0, min=-1, max=1, disabled=True, style={'description_width': 'initial'})
trained_X_widget = ipywidgets.BoundedIntText(description='X', value=0, min=0, max=height, step=1, disabled=True, style={'description_width': 'initial'})
trained_Y_widget = ipywidgets.BoundedIntText(description='Y', value=0, min=0, max=height, step=1, disabled=True, style={'description_width': 'initial'})
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress', disabled=True, style={'description_width': 'initial'})

In [7]:
# 定义播放按钮
play_button = ipywidgets.ToggleButton(description='play', value=False, disabled=False)
change_bool = play_button.value
def play():
    global image_list, change_bool
    while True:
        # if change_bool is True
        if change_bool:
            image_dict = image_list.get_next()
            set_trained_widget(image_dict)
        time.sleep(0.1)
execute_thread = threading.Thread(target=play)
execute_thread.start()

def playclick(change):
    global change_bool
    change_bool = change.new
    if play_button.description == "play":
        play_button.description = "stop"
    else:
        play_button.description = "play"
play_button.observe(playclick, names='value')

forward_button = ipywidgets.Button(description='next')
def forward(c):
    image_dict = image_list.get_next(step_widget.value)
    set_trained_widget(image_dict)
forward_button.on_click(forward)

back_button = ipywidgets.Button(description='back')
def back(c):
    image_dict = image_list.get_next(-step_widget.value)
    set_trained_widget(image_dict)
back_button.on_click(back)

step_widget = ipywidgets.BoundedIntText(value=1, min=0, max=count_widget.value, step=1)

def set_trained_widget(image_dict):
    global evaluated
    if evaluated is True:
        CH1_value, CH2_value = image_dict["trained_channel"]["CH1"], image_dict["trained_channel"]["CH2"]
        img_x, img_y = image_dict["trained_XY"]["X"], image_dict["trained_XY"]["Y"]
        image = image_list.draw_circle(image_value=image_dict["img_value"], img_x=img_x, img_y=img_y)
        train_view_widget.value = bgr8_to_jpeg(image)
        trained_X_widget.value = img_x
        trained_Y_widget.value = img_y
        trained_CH1_widget.value = CH1_value
        trained_CH2_widget.value = CH2_value

In [8]:
eval_button = ipywidgets.Button(description='evaluate', disabled=True)
evaluated = False
def evla_img_list(change):
    global image_list, model_trt, evaluated
    progress_widget.value = 0
    for single_dict in image_list.iter_img_list():
        image = single_dict["img_value"]
        preprocessed = preprocess(image).half()
        local_output = model_trt(preprocessed).detach().cpu().numpy().flatten()
        CH1 = local_output[0]
        CH2 = local_output[1]
#         print(local_output)
        img_x, img_y = channel12ToImgSize(CH1, CH2, width, height)
        image_list.json_dict[single_dict["count"]].update({
            "trained_channel":{"CH1":CH1, "CH2":CH2},
            "trained_XY":{"X":img_x, "Y":img_y}})
        progress_widget.value += 1 / len(image_list)
    evaluated = True
    progress_widget.value = 0
eval_button.on_click(evla_img_list)




In [9]:
play_botton_widget = ipywidgets.HBox([play_button,back_button,step_widget,forward_button])
trained_XY_text_widget = ipywidgets.HBox([trained_X_widget, trained_Y_widget])
trained_channel_text_widget = ipywidgets.HBox([trained_CH1_widget, trained_CH2_widget])
train_mode_right = ipywidgets.VBox([
    train_view_widget,
    play_botton_widget, 
    trained_XY_text_widget,
    trained_channel_text_widget,
    progress_widget,
    ipywidgets.HBox([eval_button])
])
display(train_mode_right)

In [10]:
eval_button.disabled = False